In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV,cross_val_score
from sklearn.metrics import confusion_matrix,classification_report,precision_score,recall_score,f1_score,plot_roc_curve

In [ ]:
pd.set_option('expand_frame_repr', False)

In [ ]:
df=pd.read_csv(r"E:\Andrei workspace\Milestone project - 2\bluebook-for-bulldozers\bluebook-for-bulldozers\TrainAndValid.csv",low_memory=False)
df.head()

In [ ]:
df.info()

In [ ]:
fig,ax=plt.subplots(figsize=(10,7))
ax.scatter(df["saledate"][:2000],df["SalePrice"][:2000]);

In [ ]:
df.SalePrice.plot.hist();

In [ ]:
df.saledate.dtype

In TS data, better parse dates

In [ ]:
df=pd.read_csv(r"E:\Andrei workspace\Milestone project - 2\bluebook-for-bulldozers\bluebook-for-bulldozers\TrainAndValid.csv",low_memory=False,parse_dates=['saledate'])
df.head()

In [ ]:
df.saledate.dtype

In [ ]:
fig,ax=plt.subplots(figsize=(10,7))
ax.scatter(df["saledate"][:2000],df["SalePrice"][:2000]);

In [ ]:
df.head().T

In [ ]:
df.saledate.head(10)

### Sort DataFrame by saledate

In [ ]:
df.sort_values(by=['saledate'],inplace=True,ascending=True)
df.saledate.head(10)

In [ ]:
df_tmp=df.copy()

In [ ]:
df_tmp.MachineID.nunique()

In [ ]:
df.shape

### Add datetime parameter for saledate column

In [ ]:
df_tmp['saleYear']=df_tmp.saledate.dt.year
df_tmp['saleMonth']=df_tmp.saledate.dt.month
df_tmp['saleDay']=df_tmp.saledate.dt.day
df_tmp['saleDayOfWeek']=df_tmp.saledate.dt.dayofweek
df_tmp['saleDayOfYear']=df_tmp.saledate.dt.dayofyear
df_tmp.head().T

sincewe enriched our data, we can remove saledate

In [ ]:
df_tmp.drop('saledate',axis=1,inplace=True)

### Check value counts of different columns

In [ ]:
df_tmp.state.value_counts()   #state ka value counts

### Done enough EDA, now model driven EDA

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)
#model.fit(df_tmp.drop("SalePrice", axis=1), df_tmp.SalePrice)
df_tmp.info()

In [ ]:
pd.api.types.is_string_dtype(df_tmp["UsageBand"])

In [ ]:
df_tmp.loc[:,df_tmp.dtypes == object].columns

In [ ]:
# These columns contain strings
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label]= content.astype('category').cat.as_ordered() ## it means, convert into category and then number those category
df_tmp.info()  ##check out, we added a new dtype category

### But despite we cant see in info, the categories are given numeric value

In [ ]:
df_tmp.state.cat.codes   # liek for datetime we had dt. for categorical types, we have cat.

In [ ]:
df_tmp.state.cat.categories

Now check ratio of missing values

In [ ]:
(df_tmp.isnull().sum()/len(df_tmp))*100

we will drop out columns where missing values are huge. But since we will start from here, lets save pre-processed data

In [ ]:
df_tmp.to_csv(r"E:\Andrei workspace\Milestone project - 2\bluebook-for-bulldozers\bluebook-for-bulldozers\train_tmp.csv",index=False)

In [ ]:
df_tmp=pd.read_csv(r"E:\Andrei workspace\Milestone project - 2\bluebook-for-bulldozers\bluebook-for-bulldozers\train_tmp.csv",low_memory=False)

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
#numeric values having null values
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content) & (pd.isnull(content).sum()>0):
            print(label)
            
        

Fill numeric row with median and  create a new column to know originality which we replaced

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content) & (pd.isnull(content).sum()>0):
            df_tmp[label+"_is_missing"]=pd.isnull(content)
            df_tmp[label]=content.fillna(content.median())   #take a note here, we did content.fillna not df['....'].fillna()
            

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content) & (pd.isnull(content).sum()>0):
            print(label)
            
## this prints out nothing because we already filled null value with median

In [ ]:
#the new columns we created
df_tmp.auctioneerID_is_missing.value_counts()

In [ ]:
# Check columns which *aren't* numeric
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)


When we turned a string column to categories, we had access to an attribute called Codes, which gave us a numerical value for all of the variables in that column. 

In [ ]:
pd.Categorical(df_tmp['state'])

In [ ]:
pd.Categorical(df_tmp['state']).dtype

In [ ]:
pd.Categorical(df_tmp['state']).codes

### Turn categorical variales into numbers 
- by converting into categorical code, null values will be appointed as -1
### convert -1 categories (the null values) into 0 by adding +1 to all

In [ ]:

for label,content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_tmp[label+"_is_missing"]=pd.isnull(content)  #in new columns, we will have true and flase
        #Turn categories into numbers and add+1, since missing values are assigned -1
        df_tmp[label]=pd.Categorical(content).codes+1  

In [ ]:
df_tmp.info()

In [ ]:
df_tmp.head().T  #lot of x_is_missing columns

In [ ]:
df_tmp.isnull().sum()  #now, we have no missing values

### Finally, no null values and all numeric! Hurray!

In [ ]:
df_tmp.head()

In [ ]:
model=RandomForestRegressor(n_jobs=-1,random_state=42)
model.fit(df_tmp.drop("SalePrice", axis=1), df_tmp.SalePrice)

In [ ]:
model.score(df_tmp.drop("SalePrice", axis=1), df_tmp.SalePrice)

Even though we used whole data, yet score isnt 100%.
### WHY?

training data set (as in data-dictionary) has data to end of 2011, validation has data from Jan 1 2012 to april 30 2012, test from May1 to nov2012. <br>
df_tmp we used is "train and val" set. So we break it into before 2012 and after. 

In [ ]:
df_tmp.saleYear.value_counts()

In [ ]:
# Split data into training and validation
df_val = df_tmp[df_tmp.saleYear == 2012]
df_train = df_tmp[df_tmp.saleYear != 2012]

df_val.shape, df_train.shape

In [ ]:
# Split data into X & y
X_train, y_train = df_train.drop("SalePrice", axis=1), df_train.SalePrice
X_valid, y_valid = df_val.drop("SalePrice", axis=1), df_val.SalePrice

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

here evaluation metric is RSML(Root sq mean error), but model.score uses coeff of determination
<br> if you see some metric like RMSE or RMSLE, take it like ratio. like 10% off. But if MAE, its like $10 off

In [ ]:
from sklearn.metrics import mean_squared_log_error,mean_absolute_error,r2_score
def rmsle(y_test, y_pred):
    return np.sqrt(mean_squared_log_error(y_test, y_pred))
# Create function to evaluate our model
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_valid, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_valid, val_preds),
              "Training R^2": r2_score(y_train, train_preds),
              "Valid R^2": r2_score(y_valid, val_preds)}
    return scores

#### Testing our model on subset to check hyperparameters

In [ ]:
# # This takes far too long... for experimenting

# %%time
# model = RandomForestRegressor(n_jobs=-1, 
#                               random_state=42)

# model.fit(X_train, y_train)
len(X_train)

In [ ]:
model = RandomForestRegressor(n_jobs=-1, 
                              random_state=42)
model.get_params()

If bootstrap is true, the number of samples to draw from X to train each base estimator.If none, then draw x shape samples, so it's going to draw them all. <br>
If bootstrap is True, the number of samples to draw from X to train each base estimator.
- If None (default), then draw X.shape[0] samples. ie its gonna draw them all
- If int, then draw max_samples samples.
- If float, then draw max_samples * X.shape[0] samples. Thus, max_samples should be in the interval (0.0, 1.0].

In [ ]:
X_train.shape[0]

id we leave max_samples as none, then every estimator (ie 100) will see every single one of X_train.shape[0]...but if max_samples=10,000 <br>
then, every estimator will see, 401125/10,000 times less. Therefore speed increase by 40 times
-  Cutting down on the max number of samples each estimator can see improves training time


In [ ]:
model = RandomForestRegressor(n_jobs=-1,random_state=42,max_samples=10000)
model.fit(X_train, y_train)

In [ ]:
show_scores(model)

In [ ]:
#takes 8 minutes, change n_iter=2

from sklearn.model_selection import RandomizedSearchCV
rf_grid = {"n_estimators": np.arange(10, 100, 10),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
           "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [10000]}

# Instantiate RandomizedSearchCV model
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                    random_state=42),
                              param_distributions=rf_grid,
                              n_iter=10,
                              cv=5,
                              verbose=True)

# Fit the RandomizedSearchCV model
rs_model.fit(X_train, y_train)

In [ ]:
rs_model.best_params_

In [ ]:
show_scores(rs_model)

Train a model with the best hyperparamters <br>
##### Note: These were found after 100 iterations of RandomizedSearchCV
in the tutors lappy, it took him few hours

In [ ]:

# Most ideal hyperparamters
ideal_model = RandomForestRegressor(n_estimators=40,
                                    min_samples_leaf=1,
                                    min_samples_split=14,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None,
                                    random_state=42) # random state so our results are reproducible

# Fit the ideal model
ideal_model.fit(X_train, y_train)


In [ ]:
show_scores(ideal_model)

Make predictions on test data

In [ ]:
df_test = pd.read_csv(r"E:\Andrei workspace\Milestone project - 2\bluebook-for-bulldozers\bluebook-for-bulldozers\Test.csv",
                      low_memory=False,
                      parse_dates=["saledate"])

df_test.head()

In [ ]:
# Make predictions on the test dataset
#test_preds = ideal_model.predict(df_test)  #error

cz has missing values, doesnt has year, month etc row, has non-numeric columns etc

Preprocessing the data (getting the test dataset in the same format as our training dataset)

In [ ]:
def preprocess_data(df):
    """
    Performs transformations on df and returns transformed df.
    """
    df['saleYear']=df.saledate.dt.year
    df['saleMonth']=df.saledate.dt.month
    df['saleDay']=df.saledate.dt.day
    df['saleDayOfWeek']=df.saledate.dt.dayofweek
    df['saleDayOfYear']=df.saledate.dt.dayofyear
    df.drop('saledate',axis=1,inplace=True)
    #fill numeric rows with median 
    for label, content in df.items():
        if pd.api.types.is_numeric_dtype(content) & (pd.isnull(content).sum()>0):
                df[label+"_is_missing"]=pd.isnull(content)
                df[label]=content.fillna(content.median()) 
    #fill cat data and turn cat data to num
        if not pd.api.types.is_numeric_dtype(content):
                df[label+"_is_missing"]=pd.isnull(content)
                df[label]= pd.Categorical(content).codes+1  #content.astype('category').cat.as_ordered() isnt needed. as line 19 fulfills it
                #nfact we use content.astype('category) only to order via cat.as_ordered()

    return df    

In [ ]:
# Process the test data 
df_test = preprocess_data(df_test)
df_test.head()

In [ ]:
# Make predictions on updated test data
test_preds = ideal_model.predict(df_test)

In [ ]:
X_train.head()

In [ ]:
# We can find how the columns differ using sets
set(X_train.columns) - set(df_test.columns)

In [ ]:
# Manually adjust df_test to have auctioneerID_is_missing column, its missing cz it didnt had missing_values
df_test["auctioneerID_is_missing"] = False
df_test.head()

In [ ]:
# Make predictions on the test data
test_preds = ideal_model.predict(df_test)
test_preds

In [ ]:
# Format predictions into the same format Kaggle is after
df_preds = pd.DataFrame()
df_preds["SalesID"] = df_test["SalesID"]
df_preds["SalesPrice"] = test_preds
df_preds

Now test dataframe has same features as training df, so ow we can predict

In [ ]:
test_preds=ideal_model.predict(df_test)
test_preds #its in array form, we need in dataframe

now formatting the way kaggle wants

In [ ]:
# Format predictions into the same format Kaggle is after
df_preds = pd.DataFrame()
df_preds["SalesID"] = df_test["SalesID"]
df_preds["SalesPrice"] = test_preds
df_preds

In [ ]:
df_preds.to_csv(r"E:\Andrei workspace\Milestone project - 2\bluebook-for-bulldozers\bluebook-for-bulldozers\we_predicted.csv",index=False)

In [ ]:
# Find feature importance of our best model
ideal_model.feature_importances_

In [ ]:
len(ideal_model.feature_importances_),df_test.shape  #therefore each colums ka correlation bata rha

In [ ]:
#we create helper function for top 20 most correlated features
def plot_feature(columns,importances,n=20):
    df=(pd.DataFrame({'features':columns,'feature_importances':importances})).sort_values('feature_importances',ascending=False).reset_index(drop=True)
    fig,ax=plt.subplots(figsize=(15,10))
    ax.barh(df['features'][:20],df['feature_importances'][:20])
    ax.set(ylabel='features',xlabel='Feature importance')
    ax.invert_yaxis()

In [ ]:
plot_feature(X_train.columns,ideal_model.feature_importances_)